In [2]:
#Using python SPAKE2 Library (ed25519 curve)
from spake2_local.spake2 import SPAKE2_A
from spake2_local.spake2 import SPAKE2_B
from hkdf import Hkdf
import hmac
from hashlib import sha256

In [3]:
shared_password = open("./key","r").read()

In [4]:
shared_password=shared_password.encode() #byte
mobile = SPAKE2_A(shared_password) #1. 공동의 비밀번호를 Scalar로 변환 (__init__)
car = SPAKE2_B(shared_password)

In [5]:
mobile_spake_message = mobile.start()
car_spake_message = car.start()
print(f"mobile_spake_message : {mobile_spake_message} / size : {len(mobile_spake_message)}")
print(f"car_spake_message : {car_spake_message} / size : {len(car_spake_message)}")

mobile_spake_message : b'm(F\x0e\x92\xffu}(\x9de\xa3\xa61\xc51!\xc7JI\xe1\xef\xef\xe5N\xac\xb0|\x84\xa7\xa84\xae' / size : 33
car_spake_message : b'v\x9c;\x95?\x1b,-<Z=\x88\xdf\x0ch\'5o\x7fv5\xac>\'\xed\xa3\xb4$\xdc\x7f"\xa9\xf7' / size : 33


In [6]:
#Generate Shared Key = xyG
mobile_shared_value = mobile.finish(car_spake_message)
car_shared_value = car.finish(mobile_spake_message)

In [7]:
def length_to_bytes(length_in_hex):
    length = length_in_hex[2:]
    if len(length)<2:
        length = "0"+length
    return length+"00"*7 #7byte padding 
#generate confirmation key 

mobile_name_length = length_to_bytes(hex(len(mobile.side)))
car_name_length = length_to_bytes(hex(len(car.side)))
mobile_spake_message_length = length_to_bytes(hex(len(mobile_spake_message)))
car_spake_message_length = length_to_bytes(hex(len(car_spake_message)))
shared_value_length = length_to_bytes(hex(len(mobile_shared_value)))
shared_password_length = length_to_bytes(hex(len(hex(mobile.pw_scalar))))

In [8]:
TT = "".join([
    mobile_name_length, # 모바일 측 이름 (Client) 길이 
    mobile.side.hex(), # 모바일 측 이름 (Client)의 hex
    car_name_length, # 자동차 측 이름 (server) 길이
    car.side.hex(), # 자동차 측 이름 server hex
    mobile_spake_message_length, # mobile 에서 보낸 spake 메시지
    mobile_spake_message.hex(), 
    car_spake_message_length, # car spake message 
    car_spake_message.hex(),
    shared_value_length, # spake 를 통해 만들어진 공유 키 길이 (양측 동일 )
    mobile_shared_value.hex(), # spake 를 통해 만들어진 공유 키 hex (양측 동일 )
    shared_password_length, # 공동 패스워드(register code)로 만든 scalar 의 길이 
    hex(mobile.pw_scalar)[2:] # 공동 패스워드(register code)로 만든 scalar 의 hex
])

In [9]:
hashed_TT = sha256(TT.encode()).hexdigest()
Ka = hashed_TT[:int(len(hashed_TT)/2)]
tk = sk = Ke = hashed_TT[int(len(hashed_TT)/2):]

In [10]:
# both parties can make TT thus, Ka, Ke
print(hashed_TT)
print(Ka, "<-- only used for key confirmation")
print(Ke, "<-- Shared Key ") 

f24b1d4576742d29aa0cf0a3df9b540fefaa41e9e5cc5950e14c6f4b1ba8f3ce
f24b1d4576742d29aa0cf0a3df9b540f <-- only used for key confirmation
efaa41e9e5cc5950e14c6f4b1ba8f3ce <-- Shared Key 


In [11]:
hk = Hkdf(salt=bytes.fromhex(''),input_key_material=bytes.fromhex(b"ConfirmationKeys".hex())) 
# HKDF 생성 -- salt 값은 공백, input 값은 "ConfirmationKeys" 문자열의 hex byte 값 
okm = hk.expand(info=bytes.fromhex(Ka)) # Key confirmaiton을 위한 Ka의 hex byte를 HKDF에 인풋으로 입력한 결과 값 
Kc = sha256(okm).hexdigest() #결과 값의 sha256 해시

KcA = Kc[:int(len(Kc)/2)] 
ConfirmationKey_Phone = hmac.new(KcA.encode(),TT.encode(),sha256).hexdigest() #해시의 앞 절반은 모바일 --> 자동차 

KcB = Kc[int(len(Kc)/2):] 
ConfirmationKey_Car = hmac.new(KcB.encode(),TT.encode(),sha256).hexdigest() #해시의 뒤 절반은 자동차 --> 모바일 

In [12]:
print(f"Kc : {Kc}")
print(f"KcA : {KcA}")
print(f"ConfirmationKey_Phone : {ConfirmationKey_Phone}")

print(f"KcB : {KcB}")
print(f"ConfirmationKey_Car : {ConfirmationKey_Car}")


Kc : 9a92a5bc3427dee0bdd491cf9cafb4ea9e87c95de4d2adff6e0fe52230df7af7
KcA : 9a92a5bc3427dee0bdd491cf9cafb4ea
ConfirmationKey_Phone : 2dffbabca02f98d5fc204f483b16863940ca5164dd5724db53e2c6d2ad72f53e
KcB : 9e87c95de4d2adff6e0fe52230df7af7
ConfirmationKey_Car : 2a673f808471e91af567351000d1a6f21fc9707f5b5a9a07e2d3c17768dcf573


In [13]:
# 모바일 측에서는 자동차에서 보낸 ConfirmationKey_B를 받아서 검증
mobile.key_confirmation(TT,ConfirmationKey_Car)

True

In [14]:
# 자동차 측에서는 모바일에서 보낸 ConfirmationKey_B를 받아서 검증
car.key_confirmation(TT,ConfirmationKey_Phone)

True

# Challenge process

In [15]:
from Crypto import Random
from Crypto.Cipher import AES

BLOCK_SIZE = 16

def pad(s):
    s = s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE).encode()
    return s

def unpad(encrypted_string):
    return encrypted_string[0:-(encrypted_string[-1])]


class AESCipher(object):
    def __init__(self, key):
        self.key = key.encode()

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        # print(f"cipher len : {len(cipher.encrypt(raw))}")
        return iv + cipher.encrypt(raw)
    
    def decrypt(self, enc):
        iv = enc[:BLOCK_SIZE]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[BLOCK_SIZE:]))

def AES_key_padding(z):
    if len(z) > 32 : 
        z = z[:32]
    else:
        for i in range(32-len(z)):
            z = z+"0"
    return z 

In [16]:
#Generate Nonce 
import binascii
import os 
import struct 
import datetime

nonce_c = int(binascii.hexlify(os.urandom(4)),16).to_bytes(4,"big")# 4byte random int
ts = struct.pack(">i",int(datetime.datetime.timestamp(datetime.datetime.now())))
plain_challenge_from_car = nonce_c+ts

### Car --> Phone Challenge

In [17]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

# todo 혼다측 요구사항 (2중암호화)을 위해 register code로 한번 더 암호화 해야됨 
# shared_password로 1차 암호화 한 후 
# spake를 통해 만들어진 Ke로 2차 암호화 

print("plain    :",plain_challenge_from_car)

encrypted_challenge_from_car1 = Car_cipher1.encrypt(plain_challenge_from_car)
print("encrypted1:",encrypted_challenge_from_car1)

encrypted_challenge_from_car2 = Car_cipher2.encrypt(encrypted_challenge_from_car1)
print("encrypted2:",encrypted_challenge_from_car2)


plain    : b'\xfe4,\xdab\xb1kz'
encrypted1: b'\x10\x12Y\xb0<$g\x11\xff^\xfa\x909w\x80\xa0\xf6 \xd7:u3\r\xf1\xed~gf\xc7\x0e7\xc4'
encrypted2: b'\x07J\x0b$\x9b!\xa3-\xe7\x80\x834\xc7\x87A\xa7\xe0\xe2\x8a\xfe\x0fa\x15\xed\x0e\x1c\x8bw\xa2\xd8n\x9f\x1a\xb5\xc0\xe7\x19\xc1C\xaf\x05\x16\xbb\xb1\xd0\xa9\xb8Q\xfftq\x1b\x81\xacp@\xc2v\x105\x9eV\\\x7f'


In [18]:
print(len(encrypted_challenge_from_car1))
print(len(encrypted_challenge_from_car2))

32
64


### Phone --> Car Challenge

In [19]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Phone_cipher1 = AESCipher(key=AES_key1)  # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Phone_cipher2 = AESCipher(key=AES_key2)  #  2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

# 2번 복호화 
decrypted_challenge_from_car1 = Phone_cipher2.decrypt(encrypted_challenge_from_car2)
decrypted_challenge_from_car = Phone_cipher1.decrypt(decrypted_challenge_from_car1)

print("decrypted:",decrypted_challenge_from_car)

decrypted: b'\xfe4,\xdab\xb1kz'


In [20]:
username = "AUTOCRYPT"
padded_username = int(username.encode().hex(),16).to_bytes(64,"big")
plain_response_from_phone = decrypted_challenge_from_car+padded_username

In [21]:
# 2중암호화 
encrypted_response_from_phone1 = Phone_cipher1.encrypt(plain_response_from_phone)
encrypted_response_from_phone2 = Phone_cipher2.encrypt(encrypted_response_from_phone1)

### Car respose check (challenge solve check)

In [22]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

#2중 복호화 
decrypted_response_from_phone1 = Car_cipher2.decrypt(encrypted_response_from_phone2)
decrypted_response_from_phone = Car_cipher1.decrypt(decrypted_response_from_phone1)

In [23]:
responsed_nonce_c = decrypted_response_from_phone[0:4]
responsed_timestamp_c = decrypted_response_from_phone[4:8]
responsed_username = decrypted_response_from_phone[8:]

In [24]:
responsed_nonce_c == nonce_c

True

In [25]:
nowtime = int(datetime.datetime.timestamp(datetime.datetime.now()))
challenge_time = int(responsed_timestamp_c.hex(),16)
elapsed_minute = (nowtime-challenge_time)/60 

In [26]:
# 2분 이내에 challenge 해결해야할 경우 
elapsed_minute < 2 

True

### Vehicle Info --> Phone 

In [27]:
# 3.1. Gen. vehicle info RPS_DATA(CMD, MODE, Clearance, ETC) 
# on Car side 

import bitstring
#通信仕様_RPSSmartphone.xlsx 참고
CMD = bitstring.BitArray(hex="0x11") #1byte 
MODE = bitstring.BitArray(hex="0x00") #1byte 
ENG_Status_Reserved = bitstring.BitArray(bin="0b0")  #1bit00
Door_Lock_Status_Reserved = bitstring.BitArray(bin="0b0")  #1bit00
Advertise_Req = bitstring.BitArray(bin="0b00")  #1bit00
Longitudinal_direction = bitstring.BitArray(bin="0b00")  #1bit00
Lateral_direction = bitstring.BitArray(bin="0b00")  #1bit00
Progress = bitstring.BitArray(hex="0x55") #1byte 0x00~0x64 (100)
clearance1 =bitstring.BitArray(bin="0b01010000") #1byte sonar 1~4 (0,1)
clearance2 = bitstring.BitArray(bin="0b01010000") #1byte sonar 5~8 (0,1)
clearance3 = bitstring.BitArray(bin="0b01010000") #1byte sonar 9~12 (0,1) 
Error_Code = bitstring.BitArray(hex="0x00") #1byte 
working_time = bitstring.BitArray(hex="0x11") #1byte
VID = bitstring.BitArray(hex="0x00000000") #4byte
SNO = bitstring.BitArray(hex="0x00") #1byte
Reserved1 = bitstring.BitArray(bin="0b0000") #4bit
Vibration_Type_Reserve = bitstring.BitArray(bin="0b000") #3bit
Vibration_timming = bitstring.BitArray(bin="0b0") #1bit
Touch_ID = bitstring.BitArray(hex="0x00") #1byte 

##### challenge는 현재 4byte --> 8 Byte 
Challenge  = bitstring.BitArray(hex="0x0000ffff0000ffff") #4byte 

VAC = bitstring.BitArray(hex="0x31") #1byte 
Reserved2 = bitstring.BitArray(bin="0b000000") #6bit 
CRC10 =  bitstring.BitArray(bin="0b0011110000") #10bit 

#모든 bit를 나열해서 붙임 
RPS_message_example = CMD+MODE+ENG_Status_Reserved+Door_Lock_Status_Reserved+Advertise_Req+Longitudinal_direction+Lateral_direction+Progress+clearance1+clearance2+clearance3+Error_Code+working_time+VID+SNO+Reserved1+Vibration_Type_Reserve+Vibration_timming+Touch_ID+Challenge+VAC+Reserved2+CRC10
from Crypto.Hash import CMAC
from Crypto.Cipher import AES

#RPS 메시지의 CMAC 생성해서 데이터 무결성 검증 
RPS_message_cmac = CMAC.new(tk.encode(), ciphermod=AES)
RPS_message_cmac.update(RPS_message_example.hex.encode())
#send this message to Phone from Car


In [28]:
# Vehicle info 2중 암호화
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

encrypted_vehicle_info1 = Car_cipher1.encrypt(RPS_message_example.hex.encode())
encrypted_vehicle_info2 = Car_cipher2.encrypt(encrypted_vehicle_info1)


In [29]:
vehicle_info_message_with_cmac = encrypted_vehicle_info2+RPS_message_cmac.digest() # 112byte + 16byte  = 128 Byte

### Check vehicle info and show info on Phone

In [30]:
# on phone side 
# 3.4 Verify MAC
RPS_message_from_car = vehicle_info_message_with_cmac[:112]
RPS_message_cmac_from_car = vehicle_info_message_with_cmac[112:]

#2중 복호화
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Phone_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Phone_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

decrypted_vehicle_info1 = Phone_cipher2.decrypt(RPS_message_from_car)
decrypted_vehicle_info = Phone_cipher1.decrypt(decrypted_vehicle_info1)

#check if CMAC is correct 
phone_cmac_confirm = CMAC.new(tk.encode(), ciphermod=AES)
phone_cmac_confirm.update(decrypted_vehicle_info)

In [31]:
phone_cmac_confirm.digest() == RPS_message_cmac_from_car

True

### generate User command

In [32]:
# on phone side
# get user input from smart phone app and generate user_command 
user_command = bitstring.BitArray(os.urandom(32)) #32byte 임의의 값 

In [33]:
#2중 암호화
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Phone_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Phone_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

encrypted_user_command1 = Phone_cipher1.encrypt(user_command.hex.encode())
encrypted_user_command2 = Phone_cipher2.encrypt(encrypted_user_command1)

In [34]:
# 3.3. Create MAC CMAC(PHONE_DATA, tk)
#user command 의 CMAC 생성해서 데이터 무결성 검증 
user_command_cmac = CMAC.new(tk.encode(), ciphermod=AES)
user_command_cmac.update(user_command.hex.encode())

In [35]:
#add CMAC to encrypted message (128byte + 16 byte = 144 byte)
phone_user_command_message  = encrypted_user_command2 + user_command_cmac.digest()

In [36]:
len(phone_user_command_message) 

144

### check User command and execute CMD

In [37]:
# on car side 
# 3.6. dec_tk(encrypted data) PHONE_DATA
recv_user_command_from_phone = phone_user_command_message

#seperate encrypted usercommand and CMAC
encrypted_user_command = recv_user_command_from_phone[:128]
user_command_cmac = recv_user_command_from_phone[128:]


#2중 복호화
AES_key1 = AES_key_padding(shared_password.decode())
AES_key2 = AES_key_padding(Ke)

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

decrypted_user_command1 = Car_cipher2.decrypt(encrypted_user_command)
decrypted_user_command = Car_cipher1.decrypt(decrypted_user_command1)


#check CMAC
# 3.3. Create MAC CMAC(PHONE_DATA, tk)
#user command 의 CMAC 생성해서 데이터 무결성 검증 
user_command_check_cmac = CMAC.new(tk.encode(), ciphermod=AES)
user_command_check_cmac.update(decrypted_user_command)

In [38]:
user_command_check_cmac.digest()

b'\x04;bR\x82\x9au\x87\xbfCF\x81\x8c\x03xh'

In [39]:
user_command_check_cmac.digest()

b'\x04;bR\x82\x9au\x87\xbfCF\x81\x8c\x03xh'

In [40]:
user_command_check_cmac.digest() == user_command_cmac

True

# on car side 
### execute CMD
